# Semantisk Kernel med OpenBnB MCP Server Integration

Denne notebook viser, hvordan man bruger Semantisk Kernel med den faktiske OpenBnB MCP-server til at søge efter rigtige Airbnb-indkvarteringer ved hjælp af MCPStdioPlugin. For adgang til LLM anvendes Azure AI Foundry. For at konfigurere dine miljøvariabler kan du følge [Setup Lesson](/00-course-setup/README.md)


## Importer de nødvendige pakker


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Oprettelse af MCP Plugin-forbindelse

Vi vil oprette forbindelse til [OpenBnB MCP-serveren](https://github.com/openbnb-org/mcp-server-airbnb) ved hjælp af MCPStdioPlugin. Denne server tilbyder søgefunktionalitet for Airbnb via @openbnb/mcp-server-airbnb-pakken.


## Oprettelse af klienten

I dette eksempel vil vi bruge Azure AI Foundry til vores LLM-adgang. Sørg for, at dine miljøvariabler er korrekt opsat.


## Miljøkonfiguration

Konfigurer Azure OpenAI-indstillinger. Sørg for, at du har følgende miljøvariabler sat:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Forståelse af OpenBnB MCP-integration

Denne notebook forbinder til **den rigtige OpenBnB MCP-server**, som leverer faktisk Airbnb-søgefunktionalitet.

### Sådan fungerer det:

1. **MCPStdioPlugin**: Bruger standard input/output-kommunikation med MCP-serveren
2. **Ægte NPM-pakke**: Downloader og kører `@openbnb/mcp-server-airbnb` via npx
3. **Live-data**: Returnerer faktiske Airbnb-ejendomsdata fra deres API'er
4. **Funktionsopdagelse**: Agenten opdager automatisk tilgængelige funktioner fra MCP-serveren

### Tilgængelige funktioner:

OpenBnB MCP-serveren tilbyder typisk:
- **search_listings** - Søg efter Airbnb-ejendomme baseret på placering og kriterier
- **get_listing_details** - Få detaljerede oplysninger om specifikke ejendomme
- **check_availability** - Tjek tilgængelighed for specifikke datoer
- **get_reviews** - Hent anmeldelser for ejendomme
- **get_host_info** - Få information om ejendomsværter

### Forudsætninger:

- **Node.js** installeret på dit system
- **Internetforbindelse** til at downloade MCP-serverpakken
- **NPX** tilgængelig (følger med Node.js)

### Test af forbindelsen:

Du kan teste MCP-serveren manuelt ved at køre:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Dette vil downloade og starte OpenBnB MCP-serveren, som Semantic Kernel derefter forbinder til for at hente reelle Airbnb-data.


## Kør agenten med OpenBnB MCP-serveren

Nu vil vi køre AI-agenten, der forbinder til OpenBnB MCP-serveren for at søge efter rigtige Airbnb-indkvarteringer i Stockholm for 2 voksne og 1 barn. Du kan frit ændre `user_inputs`-listen for at justere søgekriterierne.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Resumé
Tillykke! Du har med succes bygget en AI-agent, der integrerer med virkelige bolig-søgninger ved hjælp af Model Context Protocol (MCP):

## Anvendte teknologier:
- Semantic Kernel - Til opbygning af intelligente agenter med Azure OpenAI
- Azure AI Foundry - Til LLM-funktioner og chat-komplettering
- MCP (Model Context Protocol) - Til standardiseret værktøjsintegration
- OpenBnB MCP Server - Til ægte Airbnb-søgefunktionalitet
- Node.js/NPX - Til at køre den eksterne MCP-server

## Hvad du har lært:
- MCP-integration: Forbindelse af Semantic Kernel-agenter til eksterne MCP-servere
- Realtids dataadgang: Søgning efter faktiske Airbnb-ejendomme via live-API'er
- Protokolkommunikation: Brug af stdio-kommunikation mellem agent og MCP-server
- Funktionsopdagelse: Automatisk opdagelse af tilgængelige funktioner fra MCP-servere
- Streaming-responser: Indfangning og logning af funktionskald i realtid
- HTML-rendering: Formatering af agentens svar med stiliserede tabeller og interaktive visninger

## Næste skridt:
- Integrer yderligere MCP-servere (vejr, fly, restauranter)
- Byg et multi-agent system, der kombinerer MCP og A2A-protokoller
- Opret brugerdefinerede MCP-servere til dine egne datakilder
- Implementer vedvarende samtalehukommelse på tværs af sessioner
- Udrul agenten til Azure Functions med MCP-server orkestrering
- Tilføj brugerautentifikation og bookingfunktioner



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
